In [13]:
import pandas as pd
import sqlite3
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score, mean_poisson_deviance, mean_gamma_deviance
from sklearn.metrics import mean_squared_error, mean_absolute_error, classification_report
import xgboost as xgb

conn = sqlite3.connect(r'C:\Users\Owner\dev\algobetting\infra\data\db\algobetting.db')

df = pd.read_sql_query("""
                    SELECT DISTINCT
                        f.*,
                        ms.passing_corner_kicks as corner_kicks,
                        ms.opp_passing_corner_kicks as opp_corner_kicks
                    FROM 
                        team_all_features_365_005 f
                    JOIN
                        fbref_match_summary_v2 ms
                            ON ms.match_url = f.match_url
                            AND ms.team = f.team
                    WHERE 
                        f.team_rolling_summary_minutes IS NOT NULL
                        AND NOT (f.`is_promoted?` = 1 AND f.`is_early_season?` = 1) OR (f.`is_early_season?` = 1 AND f.`opp_team_rolling_is_promoted?` = 1)
                       """, conn)

df

,match_url,match_date,season,division,team,opp_team,is_home,gw,is_promoted?,is_early_season?,...,opp_team_rolling_conceded_defense_challenge_tackles_pct,opp_team_rolling_conceded_defense_challenges_lost,opp_team_rolling_conceded_defense_blocked_shots,opp_team_rolling_conceded_defense_blocked_passes,opp_team_rolling_conceded_defense_tackles_interceptions,opp_team_rolling_conceded_defense_clearances,opp_team_rolling_conceded_defense_errors,opp_team_rolling_conceded_keeper_psxg,corner_kicks,opp_corner_kicks
0,https://fbref.com/en/matches/e4bb1c35/Tottenha...,2025-05-25 00:00:00,2024-2025,Premier League,Tottenham,Brighton,1,38,0,0,...,50.720617,8.796946,3.393454,7.020603,26.403010,27.177737,0.862971,1.299603,2.0,11.0
1,https://fbref.com/en/matches/e4bb1c35/Tottenha...,2025-05-25 00:00:00,2024-2025,Premier League,Brighton,Tottenham,0,38,0,0,...,57.889854,7.949406,3.378971,9.378588,32.038948,23.558919,0.911136,1.805191,11.0,2.0
2,https://fbref.com/en/matches/1ff370e8/Bournemo...,2025-05-25 00:00:00,2024-2025,Premier League,Bournemouth,Leicester City,1,38,0,0,...,53.893970,6.606014,2.506894,6.904780,22.901258,21.697324,0.636969,1.811285,6.0,1.0
3,https://fbref.com/en/matches/1ff370e8/Bournemo...,2025-05-25 00:00:00,2024-2025,Premier League,Leicester City,Bournemouth,0,38,1,0,...,49.279905,7.520887,4.153870,6.746572,23.732322,26.770789,1.451179,1.460762,1.0,6.0
4,https://fbref.com/en/matches/36844e73/Newcastl...,2025-05-25 00:00:00,2024-2025,Premier League,Newcastle Utd,Everton,1,38,0,0,...,53.064874,6.895930,2.600829,6.075265,24.063503,26.142993,0.567784,1.251694,12.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39966,https://fbref.com/en/matches/e8e75a75/Como-Cag...,2022-08-13 00:00:00,2022-2023,Serie B,Como,Cagliari,1,1,0,1,...,47.905175,6.408071,3.518286,6.676135,21.826232,17.942850,0.219500,1.367898,6.0,5.0
39967,https://fbref.com/en/matches/e8e75a75/Como-Cag...,2022-08-13 00:00:00,2022-2023,Serie B,Cagliari,Como,0,1,0,1,...,42.604852,8.589310,2.172112,6.611274,23.664988,14.871051,0.075024,1.541054,5.0,6.0
39968,https://fbref.com/en/matches/99979e25/Cittadel...,2022-08-13 00:00:00,2022-2023,Serie B,Cittadella,Pisa,1,1,0,1,...,47.089937,7.713670,2.909231,6.588752,27.403947,19.741195,0.237578,1.160446,5.0,5.0
39969,https://fbref.com/en/matches/99979e25/Cittadel...,2022-08-13 00:00:00,2022-2023,Serie B,Pisa,Cittadella,0,1,0,1,...,50.507986,5.815970,3.866474,9.173457,27.363966,20.635355,0.149176,0.957865,5.0,5.0


In [2]:
final_df = df

In [3]:
final_df["outcome_under"] = np.where(final_df["corner_kicks"] < 5.5, 1, 0)

final_df

,match_url,match_date,season,division,team,opp_team,is_home,gw,is_promoted?,is_early_season?,...,opp_team_rolling_conceded_defense_challenges_lost,opp_team_rolling_conceded_defense_blocked_shots,opp_team_rolling_conceded_defense_blocked_passes,opp_team_rolling_conceded_defense_tackles_interceptions,opp_team_rolling_conceded_defense_clearances,opp_team_rolling_conceded_defense_errors,opp_team_rolling_conceded_keeper_psxg,corner_kicks,opp_corner_kicks,outcome_under
0,https://fbref.com/en/matches/e4bb1c35/Tottenha...,2025-05-25 00:00:00,2024-2025,Premier League,Tottenham,Brighton,1,38,0,0,...,8.796946,3.393454,7.020603,26.403010,27.177737,0.862971,1.299603,2.0,11.0,1
1,https://fbref.com/en/matches/e4bb1c35/Tottenha...,2025-05-25 00:00:00,2024-2025,Premier League,Brighton,Tottenham,0,38,0,0,...,7.949406,3.378971,9.378588,32.038948,23.558919,0.911136,1.805191,11.0,2.0,0
2,https://fbref.com/en/matches/1ff370e8/Bournemo...,2025-05-25 00:00:00,2024-2025,Premier League,Bournemouth,Leicester City,1,38,0,0,...,6.606014,2.506894,6.904780,22.901258,21.697324,0.636969,1.811285,6.0,1.0,0
3,https://fbref.com/en/matches/1ff370e8/Bournemo...,2025-05-25 00:00:00,2024-2025,Premier League,Leicester City,Bournemouth,0,38,1,0,...,7.520887,4.153870,6.746572,23.732322,26.770789,1.451179,1.460762,1.0,6.0,1
4,https://fbref.com/en/matches/36844e73/Newcastl...,2025-05-25 00:00:00,2024-2025,Premier League,Newcastle Utd,Everton,1,38,0,0,...,6.895930,2.600829,6.075265,24.063503,26.142993,0.567784,1.251694,12.0,3.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39901,https://fbref.com/en/matches/e8e75a75/Como-Cag...,2022-08-13 00:00:00,2022-2023,Serie B,Como,Cagliari,1,1,0,1,...,6.408071,3.518286,6.676135,21.826232,17.942850,0.219500,1.367898,6.0,5.0,0
39902,https://fbref.com/en/matches/e8e75a75/Como-Cag...,2022-08-13 00:00:00,2022-2023,Serie B,Cagliari,Como,0,1,0,1,...,8.589310,2.172112,6.611274,23.664988,14.871051,0.075024,1.541054,5.0,6.0,1
39903,https://fbref.com/en/matches/99979e25/Cittadel...,2022-08-13 00:00:00,2022-2023,Serie B,Cittadella,Pisa,1,1,0,1,...,7.713670,2.909231,6.588752,27.403947,19.741195,0.237578,1.160446,5.0,5.0,1
39904,https://fbref.com/en/matches/99979e25/Cittadel...,2022-08-13 00:00:00,2022-2023,Serie B,Pisa,Cittadella,0,1,0,1,...,5.815970,3.866474,9.173457,27.363966,20.635355,0.149176,0.957865,5.0,5.0,1


In [4]:
X = final_df.drop(columns=["team", "opp_team", "match_url", "match_date", "corner_kicks", "opp_corner_kicks", "outcome_under"])
# Use dummies for nominal categorical features
X = pd.get_dummies(X, columns=["season", "division"], drop_first=True)
y = final_df["outcome_under"]

# Use matches after a certain date as test
cutoff_date = '2024-08-01'  # adjust as needed
final_df['match_date'] = pd.to_datetime(final_df['match_date'])

train_mask = final_df['match_date'] < cutoff_date
test_mask = final_df['match_date'] >= cutoff_date

X_train = X[train_mask]
X_test = X[test_mask]
y_train = y[train_mask]
y_test = y[test_mask]  

In [5]:
y_test.shape[0]

5222

In [6]:
# Calculate scale_pos_weight (negative samples / positive samples)
scale_pos_weight =  3366 / 1856  # ≈ 1.81

# Train XGBoost classifier with class balancing
model = xgb.XGBClassifier(
    enable_categorical=True, 
    random_state=42,
    scale_pos_weight=scale_pos_weight
)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.3f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.646

Classification Report:
              precision    recall  f1-score   support

           0       0.51      0.21      0.30      1864
           1       0.67      0.89      0.76      3358

    accuracy                           0.65      5222
   macro avg       0.59      0.55      0.53      5222
weighted avg       0.61      0.65      0.60      5222



In [7]:
# Get feature importance as text
feature_importance = model.get_booster().get_score(importance_type='weight')
sorted_features = sorted(feature_importance.items(), key=lambda x: x[1], reverse=True)

print("Feature Importance (by weight):")
print("-" * 40)
for feature, importance in sorted_features:
    print(f"{feature:<25} {importance:>10}")

Feature Importance (by weight):
----------------------------------------
team_rolling_misc_crosses       46.0
team_rolling_pass_types_corner_kicks       39.0
team_rolling_pass_types_corner_kicks_out       37.0
opp_team_rolling_pass_types_corner_kicks_in       37.0
team_rolling_conceded_defense_errors       36.0
opp_team_rolling_conceded_pass_types_passes_switches       34.0
opp_team_rolling_conceded_defense_tackles_att_3rd       33.0
team_rolling_summary_minutes       32.0
opp_team_rolling_conceded_pass_types_corner_kicks       32.0
opp_team_rolling_conceded_pass_types_corner_kicks_out       32.0
team_rolling_misc_aerials_won_pct       31.0
opp_team_rolling_defense_tackles_def_3rd       31.0
team_rolling_pass_types_passes_switches       30.0
team_rolling_conceded_possession_take_ons_tackled_pct       30.0
team_rolling_conceded_pass_types_passes_switches       30.0
team_rolling_conceded_pass_types_through_balls       29.0
opp_team_rolling_misc_crosses       29.0
opp_team_rolling_misc_ae

In [10]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report, brier_score_loss, log_loss
from sklearn.model_selection import cross_val_score
import pickle
import os
import optuna
import numpy as np
from collections import Counter

# Quick toggle - set to True to run search, False to load existing model
RUN_OPTUNA_SEARCH = True

model_file = 'best_xgb_corner_model_under_optuna.pkl'

# Calculate class imbalance for scale_pos_weight baseline
class_counts = Counter(y_train)
baseline_scale_pos_weight = class_counts[0] / class_counts[1]
print(f"Class distribution: {dict(class_counts)}")
print(f"Baseline scale_pos_weight: {baseline_scale_pos_weight:.2f}")

def objective(trial):
    # STRICTER parameters for 400 features to prevent overfitting
    params = {
        'random_state': 42,
        
        # MUCH more aggressive regularization for high-dimensional data
        'max_depth': trial.suggest_int('max_depth', 3, 6),  # Shallower trees
        'learning_rate': trial.suggest_float('learning_rate', 0.02, 0.15),  # Lower learning rate
        'n_estimators': trial.suggest_int('n_estimators', 200, 800),  # More trees to compensate
        
        # Strong regularization
        'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 10.0),  # L1 regularization
        'reg_lambda': trial.suggest_float('reg_lambda', 1.0, 15.0),  # Strong L2 regularization
        
        # Aggressive feature/sample subsampling for correlation handling
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.3, 0.7),  # Use only 30-70% features
        'colsample_bylevel': trial.suggest_float('colsample_bylevel', 0.5, 0.9),  # Additional feature sampling
        'subsample': trial.suggest_float('subsample', 0.6, 0.9),  # Row sampling
        
        # Prevent overfitting on small leaf nodes
        'min_child_weight': trial.suggest_int('min_child_weight', 3, 15),
        'gamma': trial.suggest_float('gamma', 0.1, 2.0),  # Minimum split loss
        
        # Handle class imbalance
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 
                                              baseline_scale_pos_weight * 0.5, 
                                              baseline_scale_pos_weight * 2.0),
    }
    
    model = xgb.XGBClassifier(**params)
    
    # Use stratified CV to maintain class balance
    from sklearn.model_selection import StratifiedKFold
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv, 
                               scoring='neg_brier_score',  # Better for probabilistic predictions
                               n_jobs=-1)
    
    return cv_scores.mean()  # Higher is better (less negative)

if RUN_OPTUNA_SEARCH or not os.path.exists(model_file):
    print("Running Optuna optimization for UNDER 5.5 corners with 400 features...")
    print("Using aggressive regularization for high-dimensional data:")
    print("- max_depth: 3-6 (very shallow)")
    print("- reg_lambda: 1.0-15.0 (strong L2)")
    print("- colsample_bytree: 0.3-0.7 (aggressive feature sampling)")
    print("- min_child_weight: 3-15 (prevent small leaf overfitting)")
    print(f"- scale_pos_weight: around {baseline_scale_pos_weight:.2f} (handle imbalance)")
    
    # Create study with pruning for efficiency
    study = optuna.create_study(
        direction='maximize',
        pruner=optuna.pruners.MedianPruner(n_startup_trials=20, n_warmup_steps=2)
    )
    
    # More trials needed for 11 parameters + high dimensionality
    study.optimize(objective, n_trials=300, show_progress_bar=True)
    
    print(f"\nBest trial: {study.best_trial.number}")
    print(f"Best value (neg_brier_score): {study.best_value:.4f}")
    print(f"Best params: {study.best_params}")
    
    # Analyze regularization strength
    best_params = study.best_params
    print(f"\nRegularization analysis for 400 features:")
    print(f"- max_depth: {best_params['max_depth']} (target: ≤5)")
    print(f"- reg_lambda: {best_params['reg_lambda']:.2f} (target: ≥3)")
    print(f"- colsample_bytree: {best_params['colsample_bytree']:.2f} (target: ≤0.6)")
    print(f"- min_child_weight: {best_params['min_child_weight']} (target: ≥5)")
    print(f"- scale_pos_weight: {best_params['scale_pos_weight']:.2f} (baseline: {baseline_scale_pos_weight:.2f})")
    
    # Train final model
    model = xgb.XGBClassifier(**best_params, random_state=42)
    model.fit(X_train, y_train)
    
    # Save model
    with open(model_file, 'wb') as f:
        pickle.dump(model, f)
    
    print(f"\nModel saved to {model_file}")
    
else:
    print(f"Loading existing model from {model_file}")
    with open(model_file, 'rb') as f:
        model = pickle.load(f)

# Make predictions
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Comprehensive evaluation for betting model
accuracy = accuracy_score(y_test, y_pred)
brier_score = brier_score_loss(y_test, y_pred_proba)
logloss = log_loss(y_test, y_pred_proba)

print(f"\nFinal Model Performance (Under 5.5 Corners):")
print(f"Accuracy: {accuracy:.4f}")
print(f"Brier Score: {brier_score:.4f} (lower is better)")
print(f"Log Loss: {logloss:.4f} (lower is better)")
print(f"Mean predicted prob: {np.mean(y_pred_proba):.3f}")
print(f"Actual rate: {np.mean(y_test):.3f}")

print(f"\nClassification Report:")
print(classification_report(y_test, y_pred))

# Show final model parameters
print(f"\nFinal model parameters:")
final_params = model.get_params()
key_params = ['max_depth', 'learning_rate', 'n_estimators', 'reg_alpha', 'reg_lambda', 
              'colsample_bytree', 'colsample_bylevel', 'subsample', 'min_child_weight', 
              'gamma', 'scale_pos_weight']

for param in key_params:
    if param in final_params:
        value = final_params[param]
        print(f"{param}: {value}")

# Feature importance - critical for 400 features
if hasattr(model, 'feature_importances_'):
    feature_importance = model.feature_importances_
    feature_names = X_train.columns
    
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': feature_importance
    }).sort_values('importance', ascending=False)
    
    print(f"\nTop 20 Feature Importances (out of {len(feature_names)}):")
    print("=" * 70)
    for i, (_, row) in enumerate(importance_df.head(20).iterrows()):
        print(f"{i+1:2d}. {row['feature']:<50} {row['importance']:.6f}")
    
    # Show feature importance distribution
    print(f"\nFeature Importance Distribution:")
    print(f"Top 10 features account for: {importance_df.head(10)['importance'].sum():.1%} of total importance")
    print(f"Top 50 features account for: {importance_df.head(50)['importance'].sum():.1%} of total importance")
    print(f"Features with >0.001 importance: {(importance_df['importance'] > 0.001).sum()}")

# Calibration check for betting
print(f"\nCalibration Check (for betting decisions):")
prob_bins = [0, 0.4, 0.5, 0.6, 0.7, 0.8, 1.0]
for i in range(len(prob_bins)-1):
    mask = (y_pred_proba >= prob_bins[i]) & (y_pred_proba < prob_bins[i+1])
    if np.sum(mask) > 0:
        actual_rate = np.mean(y_test[mask])
        pred_rate = np.mean(y_pred_proba[mask])
        count = np.sum(mask)
        print(f"Prob {prob_bins[i]:.1f}-{prob_bins[i+1]:.1f}: {count:4d} samples, Pred={pred_rate:.3f}, Actual={actual_rate:.3f}")

[I 2025-07-17 09:10:29,373] A new study created in memory with name: no-name-f06d19a9-e093-45c0-80f4-81922a32b944


Class distribution: {0: 13073, 1: 21611}
Baseline scale_pos_weight: 0.60
Running Optuna optimization for UNDER 5.5 corners with 400 features...
Using aggressive regularization for high-dimensional data:
- max_depth: 3-6 (very shallow)
- reg_lambda: 1.0-15.0 (strong L2)
- colsample_bytree: 0.3-0.7 (aggressive feature sampling)
- min_child_weight: 3-15 (prevent small leaf overfitting)
- scale_pos_weight: around 0.60 (handle imbalance)


  0%|          | 0/300 [00:00<?, ?it/s]

[I 2025-07-17 09:10:50,545] Trial 0 finished with value: -0.21990814164364508 and parameters: {'max_depth': 3, 'learning_rate': 0.03131581311545455, 'n_estimators': 326, 'reg_alpha': 1.802293147919283, 'reg_lambda': 11.74834449901628, 'colsample_bytree': 0.333339271223295, 'colsample_bylevel': 0.8534007587955863, 'subsample': 0.8087985685980253, 'min_child_weight': 3, 'gamma': 1.8966949498601433, 'scale_pos_weight': 0.8067177759074043}. Best is trial 0 with value: -0.21990814164364508.
[I 2025-07-17 09:11:28,486] Trial 1 finished with value: -0.2305007516139257 and parameters: {'max_depth': 3, 'learning_rate': 0.043946004733602856, 'n_estimators': 685, 'reg_alpha': 6.696345924842284, 'reg_lambda': 2.6259932824755934, 'colsample_bytree': 0.35890940843350483, 'colsample_bylevel': 0.7317348175969512, 'subsample': 0.6586377011275929, 'min_child_weight': 13, 'gamma': 1.6115200442512185, 'scale_pos_weight': 0.598643584615129}. Best is trial 0 with value: -0.21990814164364508.
[I 2025-07-17 0

In [11]:
test_predictions = model.predict(X_test)
test_proba = model.predict_proba(X_test)

# Create dataframe with test set results
full_test_df = final_df.loc[X_test.index].copy()
full_test_df['predicted_is_won'] = test_predictions
full_test_df['prob_win'] = test_proba[:, 1]  # Probability of winning

full_test_df = full_test_df[["match_url", "match_date", "season", "division", 
                    "team", "opp_team", "is_home", "gw", "is_promoted?", "is_early_season?",
                    "corner_kicks", "predicted_is_won", "prob_win"]].dropna()

full_test_df.to_csv("corners_under_5.5.csv", index=False)